# SEGMENTING AND CLUSTERING NEIGHBOURHOODS IN TORONTO

### Step 1
Download all libraries we will need in this task:

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


### Step 2

Scraping the Wikipedia page for Toronto neighbourhood data and creating a dataframe with Postal Codes, Boroughs, and Neighborhoods.

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

r = requests.get(url)
soup = BeautifulSoup(r.content, 'html5lib')

#print(soup.prettify())

In [4]:
table = soup.find('table')


table_contents=[]

for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
#print(table_contents)

df = pd.DataFrame(table_contents)
print(df.head(10))

df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

  PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government
5        M9A         Etobicoke                  Islington Avenue
6        M1B       Scarborough                    Malvern, Rouge
7        M3B        North York                   Don Mills North
8        M4B         East York   Parkview Hill, Woodbine Gardens
9        M5B  Downtown Toronto          Garden District, Ryerson


### Step 3

Now that a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name is built, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In [5]:
geospacial_data = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
coordinates = pd.read_csv(geospacial_data)

df2 = pd.merge(df, coordinates, left_on = ['PostalCode'], right_on = ['Postal Code'], how = 'left')

In [6]:
df2.drop(columns = ['Postal Code'], inplace = True)
print(df2.head(10))

  PostalCode           Borough                      Neighborhood   Latitude  \
0        M3A        North York                         Parkwoods  43.753259   
1        M4A        North York                  Victoria Village  43.725882   
2        M5A  Downtown Toronto         Regent Park, Harbourfront  43.654260   
3        M6A        North York  Lawrence Manor, Lawrence Heights  43.718518   
4        M7A      Queen's Park     Ontario Provincial Government  43.662301   
5        M9A         Etobicoke                  Islington Avenue  43.667856   
6        M1B       Scarborough                    Malvern, Rouge  43.806686   
7        M3B        North York                   Don Mills North  43.745906   
8        M4B         East York   Parkview Hill, Woodbine Gardens  43.706397   
9        M5B  Downtown Toronto          Garden District, Ryerson  43.657162   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494  
5 -79.532242  
6 -79.194353  
7 -79.3521

### Step 4

We now explore and cluster the neighborhoods in Toronto. 

But first, count the number of Boroughs and Neighborhoods.

In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df2['Borough'].unique()),
        df2.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.
